## Introduction to NLP course (2017-2018).

## Homework 2.2: Chunking. Parsing with context free grammar

### Peter Weber & Jonatan Piñol

Objectives

1) Define, train and evaluate uni-gram and bi-gram HMM chunkers
- load the conll2000 corpus
- split the corpus to test and train
- define a class for unigram chunker
- define a class for bi-gram chunker. The bi-gram chunker should backoff on the unigram.
- train a unigram and a bi-gram chunker on the train corpus.
- evaluate and compare both chunkers on the test corpus

2) Create and use a simple context free grammar for syntactic parsing
- extend the CFG given in the lectures
- load the grammar in an nltk.RecursiveDescentParser
- use the parset to tag a to corpus (given)
- for each sentence, print the number of possible parses (correct answer below)

Correct number of parses for each sentence:
- “a young woman walks in the park” <- 1 parse
- “two young men smile” <- 1 parse
- “a young woman sees two men” <- 1 parse
- “sees two men a young woman” <- 0 parses
- “a young woman sees two old men in the park with a telescope” <- AT LEAST 3 parses
- “a young woman two old men in the park with a telescope sees” <- 0 parses
- “two angry men chase a woman with a telescope” <- 2 parses
- “a woman I know owns a telescope” <- 1 parse
- “a woman I know a telescope” <- 0 parses

In [2]:
# Import section
import nltk
from nltk.corpus import conll2000

In [37]:
# Class for unigram chunker
# Takes a corpus in a pos tagged an i-o-b chunk format as input
# Parses pos-tagged corpus with the parse funciton
# Given in class
class unigram_chunker(nltk.ChunkParserI):
    
    # Initialize and train the chunker
    def __init__(self, train_sents):
        # Take the pos and the iob tags of the corpus
        # Ignore the actual words, we map from pos tag to iob tag
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)] for sent in train_sents]
        # Train an unigram tagger from the train data
        self.tagger = nltk.UnigramTagger(train_data)
    
    # Parse function
    # Takes a corpus in POS tagged format
    def parse(self,sentence):
        # Take the pos tags
        pos_tags = [pos for (word,pos) in sentence]
        # Use the tagger to tag the modified corpus
        tagged_pos_tags = self.tagger.tag(pos_tags)
        # Take the chunks from the tagged corpus
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        # Convert the output
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag) in zip(sentence, chunktags)]
        
        # Return the tagged sentence
        return nltk.chunk.conlltags2tree(conlltags)             

In [38]:
# Class for bigram chunker
# Takes a corpus in a pos tagged an i-o-b chunk format as input
# Parses pos-tagged corpus with the parse funciton
class bigram_chunker(nltk.ChunkParserI):
    
    # Initialize and train the chunker
    def __init__(self, train_sents):
        # Take the pos and the iob tags of the corpus
        # Ignore the actual words, we map from pos tag to iob tag
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)] for sent in train_sents]
        # Train an unigram tagger from the train data
        unigram_tagger = nltk.UnigramTagger(train_data)
        self.tagger = nltk.BigramTagger(train_data, backoff = unigram_tagger)
        
    # Parse function
    # Takes a corpus in POS tagged format
    def parse(self,sentence):
        # Take the pos tags
        pos_tags = [pos for (word,pos) in sentence]
        # Use the tagger to tag the modified corpus
        tagged_pos_tags = self.tagger.tag(pos_tags)
        # Take the chunks from the tagged corpus
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        # Convert the output
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag) in zip(sentence, chunktags)]
        
        # Return the tagged sentence
        return nltk.chunk.conlltags2tree(conlltags) 
    

In [39]:
# Dummy function for exercise 1
def hw22_ex1():
    # Get the corpus
    train = conll2000.chunked_sents("train.txt")
    test = conll2000.chunked_sents("test.txt")
    
    # Train the two taggers:
    uni_chunker = unigram_chunker(train)
    bi_chunker = bigram_chunker(train)
    
    # Evaluate and print the results:
    print ("The performance of unigram chunker is: {}".format(uni_chunker.evaluate(test)))
    print ("The performance of bigram chunker is: {}".format(bi_chunker.evaluate(test)))

In [44]:
# Dummy function for exercise 2
def hw22_ex2(num_sent, plot_tree):
    """Function for exercise 2"""
    # corpus (given)
    corpus = [['a', 'young', 'woman', 'walks', 'in', 'the', 'park'], 
              ['two', 'young', 'men', 'smile'], 
              ['a', 'young', 'woman', 'sees', 'two', 'men'], 
              ['sees', 'two', 'men', 'a', 'young', 'woman'], 
              ['a', 'young', 'woman', 'sees', 'two', 'old', 'men', 'in', 'the', 'park', 'with', 'a', 'telescope'], 
              ['a', 'young', 'woman', 'two', 'old', 'men', 'in', 'the', 'park', 'with', 'a', 'telescope', 'sees'], 
              ['two', 'angry', 'men', 'chase', 'a', 'woman', 'with', 'a', 'telescope'], 
              ['a', 'woman', 'I', 'know', 'owns', 'a', 'telescope'], 
              ['a', 'woman', 'I', 'know', 'a', 'telescope']]

    # Grammar (in a string format)
    grammar_string = """
    S -> NP VP
    VP -> V NP | V NP PP | V P NP | V | V Det Nom
    PP -> P NP
    SBAR -> NP VP
    V -> "saw" | "ate" | "walked" | "walks" | "smile" | "sees" | "chase" | "owns" | "know"
    Nom -> Adj N | "I"
    NP -> "John" | "Mary" | "Bob" | Det N | Det N PP | Det Nom | Det N NP | Nom VP | Det Nom PP
    Det -> "a" | "an" | "the" | "my" | "two"
    N -> "man" | "dog" | "cat" | "telescope" | "park" | "woman" | "men"
    Adj -> "young" | "old" | "angry"
    P -> "in" | "on" | "by" | "with"
    """
    
    # Grammar (in nltk CFG format)
    grammar = nltk.CFG.fromstring(grammar_string)
    
    # Parse the corpus, 
    # count the number of parses for each sentence,
    # and print the sentence and the number of parses
    
    rd_parser = nltk.RecursiveDescentParser(grammar)
    # Print all possible trees
    sent = corpus[num_sent]
    print(sent)
    for tree in rd_parser.parse(sent):
        print(tree)

    # Draw all possible trees
    if plot_tree:
        for tree in rd_parser.parse(sent):
            tree.draw()

In [49]:
def main():
    print("\n------------------------------------------------------------------------")
    print("Exercise 1: unigram and bigram chunker")
    hw22_ex1()
    print("------------------------------------------------------------------------")
    print("\n------------------------------------------------------------------------")
    print("Exercise 2: number of parses with a CFG")
    for i in range(9):
        print("\nNext sentence")
        hw22_ex2(i, False)

In [50]:
# Running the main function
if __name__=="__main__":
    main()


------------------------------------------------------------------------
Exercise 1: unigram and bigram chunker
The performance of unigram chunker is: ChunkParse score:
    IOB Accuracy:  86.5%%
    Precision:     74.3%%
    Recall:        86.4%%
    F-Measure:     79.9%%
The performance of bigram chunker is: ChunkParse score:
    IOB Accuracy:  89.5%%
    Precision:     81.1%%
    Recall:        86.4%%
    F-Measure:     83.7%%
------------------------------------------------------------------------

------------------------------------------------------------------------
Exercise 2: number of parses with a CFG

Next sentence
['a', 'young', 'woman', 'walks', 'in', 'the', 'park']
(S
  (NP (Det a) (Nom (Adj young) (N woman)))
  (VP (V walks) (P in) (NP (Det the) (N park))))

Next sentence
['two', 'young', 'men', 'smile']
(S (NP (Det two) (Nom (Adj young) (N men))) (VP (V smile)))

Next sentence
['a', 'young', 'woman', 'sees', 'two', 'men']
(S
  (NP (Det a) (Nom (Adj young) (N woman)))
